#### Instalando as bibliotecas necessárias

In [ ]:
%pip install -q -r requirements.txt

#### Importando as bibliotecas

In [1]:
# Importando tudo que a análise econometrica precisa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pyreadstat
import seaborn as sns
import pandasql as ps

#### Importando os dados em formato sav

In [2]:
# Lê os arquivos .sav
df_general, meta1 = pyreadstat.read_sav('data/Deidentified_WPH003a Economy (General Population) (SPSS Version).sav')
df_economists, meta2 = pyreadstat.read_sav('data/Deidentified_WPH003b Economy (Economists) (SPSS Version).sav')

# Função para criar um DataFrame com labels
def create_label_df(df, meta):
    df_labels = df.copy()  # Copia o DataFrame original
    for column in df_labels.columns:
        if column in meta.variable_value_labels:  # Verifica se há rótulos para a coluna
            labels_dict = meta.variable_value_labels[column]  # Dicionário de rótulos {valor_numérico: label}
            df_labels[column] = df_labels[column].map(labels_dict)  # Mapeia os códigos para rótulos
    return df_labels

# Cria DataFrames com labels para ambos os arquivos
df_general_labels = create_label_df(df_general, meta1)
df_economists_labels = create_label_df(df_economists, meta2)

In [4]:
# Criando um dicionário com os rótulos das colunas para o público geral
dicionario_general = pd.DataFrame(meta2.column_names_to_labels.items(), columns=['coluna', 'label'])

dicionario_general

,coluna,label
0,intv,None
1,q901,PARTY CONSIDER SELF
2,q905,REGISTERED TO VOTE
3,q908,"LIBERAL,MODERATE,CONSERVATIVE"
4,q910,YEAR BORN
...,...,...
82,pri,None
83,sic,None
84,title,None
85,effort,None


In [6]:
# Criando um dicionário com os rótulos das colunas para os economistas
dicionario_economists = pd.DataFrame(meta2.column_names_to_labels.items(), columns=['coluna', 'label'])

dicionario_economists

,coluna,label
0,intv,None
1,q901,PARTY CONSIDER SELF
2,q905,REGISTERED TO VOTE
3,q908,"LIBERAL,MODERATE,CONSERVATIVE"
4,q910,YEAR BORN
...,...,...
82,pri,None
83,sic,None
84,title,None
85,effort,None


In [7]:
# Renomear colunas de df_general com o prefixo "cod_"
df_general.columns = ['cod_' + col for col in df_general.columns]


# unindo descrição de resposta (df_general_excel) com resposta codificada (df_general) usando pandasql

query = '''
SELECT 
    df_general_labels.*, df_general.*
FROM
    df_general_labels
JOIN
    df_general
ON
    df_general_labels.intv = df_general.cod_intv
'''

df_general_union = ps.sqldf(query, locals())

df_general_union.head()

,intv,q901,q905,q908,q910,q918,race,sex,state,region,...,cod_q46,cod_q47,cod_q48,cod_q49,cod_q50,cod_q51,cod_q55,cod_q56,cod_q919,cod_wt1
0,1.0,DEMOCRAT,YES,LIBERAL,None,$25000-29999,WHITE,FEMALE,37.0,SOUTH,...,6.0,NaN,2.0,NaN,NaN,2.0,5.0,6.0,2.0,0.89
1,2.0,REPUBLICAN,NO,VERY CONSERVATIVE,None,$50000-74999,WHITE,FEMALE,37.0,SOUTH,...,1.0,1.0,1.0,1.0,1.0,1.0,6.0,7.0,2.0,1.00
2,3.0,DEMOCRAT,NO,MODERATE,None,$10000-19999,WHITE,FEMALE,27.0,SOUTH,...,NaN,NaN,2.0,NaN,NaN,2.0,3.0,NaN,2.0,1.23
3,4.0,INDEPENDENT,YES,MODERATE,None,$50000-74999,WHITE,FEMALE,37.0,SOUTH,...,1.0,1.0,2.0,NaN,NaN,2.0,5.0,5.0,2.0,0.88
4,5.0,INDEPENDENT,YES,MODERATE,None,$50000-74999,WHITE,MALE,35.0,SOUTH,...,1.0,1.0,2.0,NaN,NaN,1.0,5.0,5.0,2.0,0.94


In [8]:
# Renomear colunas de df_economists com o prefixo "cod_"
df_economists.columns = ['cod_' + col for col in df_economists.columns]

# unindo descrição de resposta (df_general_excel) com resposta codificada (df_general) usando pandasql

query = '''
SELECT 
    df_economists_labels.*, df_economists.*
FROM

    df_economists_labels
JOIN
    df_economists
ON
    df_economists_labels.intv = df_economists.cod_intv
'''

df_economists_union = ps.sqldf(query, locals())

df_economists_union

,intv,q901,q905,q908,q910,q918,race,sex,state,region,...,cod_q44,cod_q45,cod_q919,cod_q115,cod_seq,cod_pri,cod_sic,cod_title,cod_effort,cod_expdate
0,1.0,None,None,None,None,None,None,None,24.0,SOUTH,...,NaN,NaN,NaN,NaN,1197.0,3.0,0.0,99.0,,9706.0
1,2.0,INDEPENDENT,YES,MODERATE,None,$100000 OR MORE,WHITE,MALE,24.0,SOUTH,...,1.0,3.0,2.0,2.0,1028.0,0.0,0.0,0.0,N50301,9704.0
2,3.0,INDEPENDENT,YES,MODERATE,None,$75000-99999,WHITE,MALE,24.0,SOUTH,...,1.0,3.0,2.0,2.0,845.0,0.0,0.0,99.0,,9706.0
3,4.0,DEMOCRAT,YES,MODERATE,None,$50000-74999,WHITE,MALE,24.0,SOUTH,...,2.0,NaN,2.0,2.0,836.0,0.0,0.0,99.0,,9612.0
4,5.0,None,None,None,None,None,None,None,24.0,SOUTH,...,NaN,NaN,NaN,NaN,933.0,0.0,0.0,99.0,,9703.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,595.0,REPUBLICAN,YES,MODERATE,None,$50000-74999,WHITE,MALE,10.0,MIDWEST,...,1.0,1.0,2.0,2.0,1966.0,0.0,5.0,99.0,,9709.0
595,596.0,None,None,None,None,None,None,None,8.0,EAST,...,NaN,NaN,NaN,NaN,332.0,0.0,0.0,99.0,,9612.0
596,597.0,None,None,None,None,None,None,None,23.0,SOUTH,...,NaN,NaN,NaN,NaN,1367.0,0.0,0.0,99.0,,9706.0
597,598.0,REPUBLICAN,YES,CONSERVATIVE,None,$100000 OR MORE,WHITE,MALE,49.0,WEST,...,2.0,NaN,2.0,2.0,2830.0,0.0,0.0,99.0,,9706.0


In [9]:
# criando as variáveis de controle

query = '''
SELECT
      df_general_union.*
    , 0 as econ
    , CASE
          WHEN cod_sex = 1
          THEN 1
          ELSE 0
      END AS male
    , 1996 - q910 AS age_in_1996
    , CASE
          WHEN cod_race NOT IN (1, 2, 3)
          THEN 1
          ELSE 0
      END AS othrace
    , CASE
          WHEN cod_race = 1
          THEN 1
          ELSE 0
      END AS white
    , CASE
          WHEN cod_race = 3
          THEN 1
          ELSE 0
      END AS asian
    , CASE
          WHEN cod_race = 2
          THEN 1
          ELSE 0
      END AS black
    , CASE
          WHEN cod_q36 = 4 THEN 3
          WHEN cod_q36 = 3 THEN 2
          WHEN cod_q36 = 2 THEN 1
          WHEN cod_q36 = 1 THEN 0
          ELSE NULL
      END AS jobsecurity
    , CASE
          WHEN cod_q15 = 3 THEN 0
          WHEN cod_q15 = 2 THEN 1
          WHEN cod_q15 = 1 THEN 2
          ELSE NULL
      END AS yourlast5
    , CASE
          WHEN cod_q17 = 2 THEN 0
          WHEN cod_q17 = 3 THEN 1
          WHEN cod_q17 = 1 THEN 2
          ELSE NULL
      END AS yournext5
    , CASE
          WHEN cod_q918 = 1 THEN 1
          WHEN cod_q918 = 2 THEN 2
          WHEN cod_q918 = 3 THEN 3
          WHEN cod_q918 = 4 THEN 4
          WHEN cod_q918 = 5 THEN 5
          WHEN cod_q918 = 6 THEN 6
          WHEN cod_q918 = 7 THEN 7
          WHEN cod_q918 = 8 THEN 8
          WHEN cod_q918 = 9 THEN 9
          ELSE NULL
      END AS income
    , CASE
          WHEN cod_q901 = 1 THEN 1
          ELSE 0
      END AS dem
    , CASE
          WHEN cod_q901 = 2 THEN 1
          ELSE 0
      END AS rep
    , CASE
          WHEN cod_q901 = 3 THEN 1
          ELSE 0
      END AS indep
    , CASE
          WHEN cod_q901 = 4 THEN 1
          ELSE 0
      END AS othparty
    , CASE
          WHEN cod_q908 = 1 THEN -2
          WHEN cod_q908 = 2 THEN -1
          WHEN cod_q908 = 3 THEN 0
          WHEN cod_q908 = 4 THEN 1
          WHEN cod_q908 = 5 THEN 2
          ELSE NULL
      END AS ideology
    , CASE
          WHEN cod_q908 = 6 THEN 1
          ELSE 0
      END AS othideol
    , CASE
          WHEN q55 = 'None' THEN 1
          WHEN cod_q55 = 1 THEN 1
          WHEN cod_q55 = 2 THEN 2
          WHEN cod_q55 = 3 THEN 3
          WHEN cod_q55 = 4 THEN 4
          WHEN cod_q55 = 5 THEN 5
          WHEN cod_q55 = 6 THEN 6
          WHEN cod_q55 = 7 THEN 7
          ELSE NULL
      END AS education
FROM
    df_general_union
'''

df_general_union = ps.sqldf(query, locals())

df_general_union

,intv,q901,q905,q908,q910,q918,race,sex,state,region,...,yourlast5,yournext5,income,dem,rep,indep,othparty,ideology,othideol,education
0,1.0,DEMOCRAT,YES,LIBERAL,None,$25000-29999,WHITE,FEMALE,37.0,SOUTH,...,0.0,0.0,4.0,1,0,0,0,-1.0,0,5.0
1,2.0,REPUBLICAN,NO,VERY CONSERVATIVE,None,$50000-74999,WHITE,FEMALE,37.0,SOUTH,...,2.0,2.0,7.0,0,1,0,0,2.0,0,6.0
2,3.0,DEMOCRAT,NO,MODERATE,None,$10000-19999,WHITE,FEMALE,27.0,SOUTH,...,2.0,1.0,2.0,1,0,0,0,0.0,0,3.0
3,4.0,INDEPENDENT,YES,MODERATE,None,$50000-74999,WHITE,FEMALE,37.0,SOUTH,...,1.0,1.0,7.0,0,0,1,0,0.0,0,5.0
4,5.0,INDEPENDENT,YES,MODERATE,None,$50000-74999,WHITE,MALE,35.0,SOUTH,...,2.0,1.0,7.0,0,0,1,0,0.0,0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,1506.0,INDEPENDENT,NO,LIBERAL,None,$30000-39999,WHITE,MALE,30.0,SOUTH,...,0.0,1.0,5.0,0,0,1,0,-1.0,0,3.0
1506,1507.0,DEMOCRAT,YES,MODERATE,None,$50000-74999,WHITE,FEMALE,25.0,SOUTH,...,0.0,1.0,7.0,1,0,0,0,0.0,0,5.0
1507,1508.0,INDEPENDENT,NO,MODERATE,None,$10000-19999,None,MALE,33.0,SOUTH,...,0.0,2.0,2.0,0,0,1,0,0.0,0,6.0
1508,1509.0,DEMOCRAT,YES,LIBERAL,None,$40000-49999,BLACK,FEMALE,26.0,SOUTH,...,1.0,2.0,6.0,1,0,0,0,-1.0,0,3.0


In [10]:
# criando as variáveis de controle

query = '''
SELECT
      df_economists_union.*
    , 1 as econ
    , CASE
          WHEN cod_sex = 1
          THEN 1
          ELSE 0
      END AS male
    , 1996 - q910 AS age_in_1996
    , CASE
          WHEN cod_race NOT IN (1, 2, 3)
          THEN 1
          ELSE 0
      END AS othrace
    , CASE
          WHEN cod_race = 1
          THEN 1
          ELSE 0
      END AS white
    , CASE
          WHEN cod_race = 3
          THEN 1
          ELSE 0
      END AS asian
    , CASE
          WHEN cod_race = 2
          THEN 1
          ELSE 0
      END AS black
    , CASE
          WHEN cod_q39 = 4 THEN 3
          WHEN cod_q39 = 3 THEN 2
          WHEN cod_q39 = 2 THEN 1
          WHEN cod_q39 = 1 THEN 0
          ELSE NULL
      END AS jobsecurity
    , CASE
          WHEN cod_q17 = 3 THEN 0
          WHEN cod_q17 = 2 THEN 1
          WHEN cod_q17 = 1 THEN 2
          ELSE NULL
      END AS yourlast5
    , CASE
          WHEN cod_q19 = 2 THEN 0
          WHEN cod_q19 = 3 THEN 1
          WHEN cod_q19 = 1 THEN 2
          ELSE NULL
      END AS yournext5
    , CASE
          WHEN cod_q918 = 1 THEN 1
          WHEN cod_q918 = 2 THEN 2
          WHEN cod_q918 = 3 THEN 3
          WHEN cod_q918 = 4 THEN 4
          WHEN cod_q918 = 5 THEN 5
          WHEN cod_q918 = 6 THEN 6
          WHEN cod_q918 = 7 THEN 7
          WHEN cod_q918 = 8 THEN 8
          WHEN cod_q918 = 9 THEN 9
          ELSE NULL
      END AS income
    , CASE
          WHEN cod_q901 = 1 THEN 1
          ELSE 0
      END AS dem
    , CASE
          WHEN cod_q901 = 2 THEN 1
          ELSE 0
      END AS rep
    , CASE
          WHEN cod_q901 = 3 THEN 1
          ELSE 0
      END AS indep
    , CASE
          WHEN cod_q901 = 4 THEN 1
          ELSE 0
      END AS othparty
    , CASE
          WHEN cod_q908 = 1 THEN -2
          WHEN cod_q908 = 2 THEN -1
          WHEN cod_q908 = 3 THEN 0
          WHEN cod_q908 = 4 THEN 1
          WHEN cod_q908 = 5 THEN 2
          ELSE NULL
      END AS ideology
    , CASE
          WHEN cod_q908 = 6 THEN 1
          ELSE 0
      END AS othideol
    , 7 as education
FROM
    df_economists_union
'''

df_economists_union = ps.sqldf(query, locals())

df_economists_union

,intv,q901,q905,q908,q910,q918,race,sex,state,region,...,yourlast5,yournext5,income,dem,rep,indep,othparty,ideology,othideol,education
0,1.0,None,None,None,None,None,None,None,24.0,SOUTH,...,NaN,NaN,NaN,0,0,0,0,NaN,0,7
1,2.0,INDEPENDENT,YES,MODERATE,None,$100000 OR MORE,WHITE,MALE,24.0,SOUTH,...,2.0,2.0,9.0,0,0,1,0,0.0,0,7
2,3.0,INDEPENDENT,YES,MODERATE,None,$75000-99999,WHITE,MALE,24.0,SOUTH,...,0.0,0.0,8.0,0,0,1,0,0.0,0,7
3,4.0,DEMOCRAT,YES,MODERATE,None,$50000-74999,WHITE,MALE,24.0,SOUTH,...,1.0,1.0,7.0,1,0,0,0,0.0,0,7
4,5.0,None,None,None,None,None,None,None,24.0,SOUTH,...,NaN,NaN,NaN,0,0,0,0,NaN,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,595.0,REPUBLICAN,YES,MODERATE,None,$50000-74999,WHITE,MALE,10.0,MIDWEST,...,2.0,2.0,7.0,0,1,0,0,0.0,0,7
595,596.0,None,None,None,None,None,None,None,8.0,EAST,...,NaN,NaN,NaN,0,0,0,0,NaN,0,7
596,597.0,None,None,None,None,None,None,None,23.0,SOUTH,...,NaN,NaN,NaN,0,0,0,0,NaN,0,7
597,598.0,REPUBLICAN,YES,CONSERVATIVE,None,$100000 OR MORE,WHITE,MALE,49.0,WEST,...,1.0,1.0,9.0,0,1,0,0,1.0,0,7


In [18]:
common_columns = df_general_union.columns.intersection(df_economists_union.columns)

common_columns

Index(['intv', 'q901', 'q905', 'q908', 'q910', 'q918', 'race', 'sex', 'state',
       'region',
       ...
       'yourlast5', 'yournext5', 'income', 'dem', 'rep', 'indep', 'othparty',
       'ideology', 'othideol', 'education'],
      dtype='object', length=110)

In [21]:
common_columns = df_general_union.columns.intersection(df_economists_union.columns)

df_general_common = df_general_union[common_columns]
df_economists_common = df_economists_union[common_columns]

# Concatenar os DataFrames
df_union = pd.concat([df_general_common, df_economists_common], ignore_index=True)

df_union

C:\Users\guian\AppData\Local\Temp\ipykernel_38644\1373302143.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_union = pd.concat([df_general_common, df_economists_common], ignore_index=True)


,intv,q901,q905,q908,q910,q918,race,sex,state,region,...,yourlast5,yournext5,income,dem,rep,indep,othparty,ideology,othideol,education
0,1.0,DEMOCRAT,YES,LIBERAL,None,$25000-29999,WHITE,FEMALE,37.0,SOUTH,...,0.0,0.0,4.0,1,0,0,0,-1.0,0,5.0
1,2.0,REPUBLICAN,NO,VERY CONSERVATIVE,None,$50000-74999,WHITE,FEMALE,37.0,SOUTH,...,2.0,2.0,7.0,0,1,0,0,2.0,0,6.0
2,3.0,DEMOCRAT,NO,MODERATE,None,$10000-19999,WHITE,FEMALE,27.0,SOUTH,...,2.0,1.0,2.0,1,0,0,0,0.0,0,3.0
3,4.0,INDEPENDENT,YES,MODERATE,None,$50000-74999,WHITE,FEMALE,37.0,SOUTH,...,1.0,1.0,7.0,0,0,1,0,0.0,0,5.0
4,5.0,INDEPENDENT,YES,MODERATE,None,$50000-74999,WHITE,MALE,35.0,SOUTH,...,2.0,1.0,7.0,0,0,1,0,0.0,0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2104,595.0,REPUBLICAN,YES,MODERATE,None,$50000-74999,WHITE,MALE,10.0,MIDWEST,...,2.0,2.0,7.0,0,1,0,0,0.0,0,7.0
2105,596.0,None,None,None,None,None,None,None,8.0,EAST,...,NaN,NaN,NaN,0,0,0,0,NaN,0,7.0
2106,597.0,None,None,None,None,None,None,None,23.0,SOUTH,...,NaN,NaN,NaN,0,0,0,0,NaN,0,7.0
2107,598.0,REPUBLICAN,YES,CONSERVATIVE,None,$100000 OR MORE,WHITE,MALE,49.0,WEST,...,1.0,1.0,9.0,0,1,0,0,1.0,0,7.0


In [22]:
df_union[['econ']]

,econ
0,0
1,0
2,0
3,0
4,0
...,...
2104,1
2105,1
2106,1
2107,1


### preparando o df para as regressões

In [ ]:
# selecionando somente as colunas que serão utilizadas na análise

df_union = df_union[[ ''
                    , ''
                    , ''
                    ]]

# escrevendo um def para estimar o modelo de logit ordenado com as variaveis de controle
# vamos usar uma formula igual o R

# selecionando apenas as perguntas que serão utilizadas na análise
questions = df_union[['']]

def logit_ordenado(df, y, x):
    x = sm.add_constant(x)
    model = sm.MNLogit(y, x)
    results = model.fit()
    return results